In [87]:
import vrep
import time
import numpy as np
from math import cos, sin
from scipy.linalg import expm,logm
from copy import copy as copy

In [65]:
def handle_grabber(i,j):
    result, handle = vrep.simxGetObjectHandle(clientID, f'hexa_joint{i+1}_{j}', vrep.simx_opmode_blocking)
    if result != vrep.simx_return_ok:
        raise Exception(f'could not get object handle for joint {i+1} of leg {j}')
    return handle
def init_handles():
    body_handles = [0,0,0,0,0,0,0]
    handles = [[handle_grabber(i,j) for i in range(3)] for j in range(6)]
    
    result, body_handles[6] = vrep.simxGetObjectHandle(clientID, 'hexa_body', vrep.simx_opmode_blocking)
    if result != vrep.simx_return_ok:
        raise Exception(f'could not get object handle for hexa body')
        
    for i in range(6):
        result, body_handles[i] = vrep.simxGetObjectHandle(clientID, f'hexa_footTip{i}', vrep.simx_opmode_blocking)
        if result != vrep.simx_return_ok:
            raise Exception(f'could not get object handle for foot tip {i+1} ')
    print(handles,body_handles)
    return handles, body_handles

In [97]:
def leg_IK(pos):
    x = pos[0]
    y = pos[1]
    z = pos[2]
    l1 = 0
    l2 = 0.25361320760487255
    l3 = 1
    we = 0.11651167932822327
    h1 = 0
    de = 0
    
    l2 = 0.05149697649731044
    l3 = 0.07238155732083357
    le = 0.11651167932822327
    phi = np.radians(0)
#     x = 2.2
#     y = -2.2
#     z = -.7

    le = np.sqrt(we*we+de*de)
    
    theta1 = np.arctan(x/np.abs(y))
    L1 = np.sqrt(x*x+y*y)
    L2 = (L1-l2)*(L1-l2)
    L = np.sqrt(z*z+L2)
    alpha1 = np.abs(np.arctan((L1-l2)/z))
    alpha2 = np.arccos((L*L+l3*l3-le*le)/(2*L*l3))
    theta2 = alpha1+alpha2-np.pi/2
    beta = np.arccos((le*le+l3*l3-L*L)/(2*le*l3))
    gamma = np.arctan(de/we)
    theta3 = beta-gamma
    return [theta1,-theta2,theta3]

def move_leg(thetas,leg):
    vrep.simxSetJointTargetPosition(clientID, handles[leg][0], thetas[0], vrep.simx_opmode_oneshot)
    vrep.simxSetJointTargetPosition(clientID, handles[leg][1], thetas[1], vrep.simx_opmode_oneshot)
    vrep.simxSetJointTargetPosition(clientID, handles[leg][2], thetas[2], vrep.simx_opmode_oneshot)
def step_3(pos):
    #starting positions
    thetas1 = leg_IK([0.13086048,-0.13086048,-0.03120471]) #0.13086048,-0.13086048,-0.03120471
    thetas2 = copy(thetas1)
    thetas3 = copy(thetas1)
    thetas4 = copy(thetas1)
    #lift up 
    thetas2[0] = thetas1[0]
    thetas2[1] = thetas1[1]
    thetas2[2] = thetas1[2]+.4
    #lift up 
    thetas3[0] = 0
    thetas3[1] = thetas1[1]
    thetas3[2] = thetas1[2]+.4
    #
    thetas4[0] = 0
    thetas4[1] = thetas1[1]
    thetas4[2] = thetas1[2]
    
    print(thetas1,thetas2,thetas3)
    print('step')
    move_leg(thetas2,0)
    move_leg(thetas2,2)
    move_leg(thetas2,4)
    move_leg(thetas4,1)
    move_leg(thetas4,3)
    move_leg(thetas4,5)
    time.sleep(.5)
    move_leg(thetas3,0)
    move_leg(thetas3,2)
    move_leg(thetas3,4)
    move_leg(thetas1,1)
    move_leg(thetas1,3)
    move_leg(thetas1,5)
    time.sleep(.5)

    move_leg(thetas4,0)
    move_leg(thetas4,2)
    move_leg(thetas4,4)
    move_leg(thetas2,1)
    move_leg(thetas2,3)
    move_leg(thetas2,5)
    time.sleep(.5)
    move_leg(thetas1,0)
    move_leg(thetas1,2)
    move_leg(thetas1,4)
    move_leg(thetas3,1)
    move_leg(thetas3,3)
    move_leg(thetas3,5)
    time.sleep(.5)


In [62]:
def get_joint(handles, body_handles):
    X = []
    Y = []
    Z = []
    result,vector=vrep.simxGetObjectPosition(clientID, handles[0][0],body_handles[6],vrep.simx_opmode_blocking)
    X.append(vector[0])
    Y.append(vector[1])
    Z.append(vector[2])
    print(vector)
    result,vector=vrep.simxGetObjectPosition(clientID, handles[0][1],body_handles[6],vrep.simx_opmode_blocking)
    X.append(vector[0])
    Y.append(vector[1])
    Z.append(vector[2])
    print(vector)
    result,vector=vrep.simxGetObjectPosition(clientID, handles[0][2],body_handles[6],vrep.simx_opmode_blocking)
    X.append(vector[0])
    Y.append(vector[1])
    Z.append(vector[2])
    print(vector)
    result,vector=vrep.simxGetObjectPosition(clientID, body_handles[0],body_handles[6],vrep.simx_opmode_blocking)
    X.append(vector[0])
    Y.append(vector[1])
    Z.append(vector[2])
    print(vector)

#     X = np.round(X, decimals = 6)
#     Y = np.round(Y, decimals = 6)
#     Z = np.round(Z, decimals = 6)
    return X,Y,Z

In [95]:
# ======================================================================================================= #
# ======================================= Start Simulation ============================================== #
# ======================================================================================================= #

# Close all open connections (Clear bad cache)
vrep.simxFinish(-1)
# Connect to V-REP (raise exception on failure)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
if clientID == -1:
	raise Exception('Failed connecting to remote API server')

# ======================================== Setup "handle"  =========================================== #
handles, body_handles = init_handles()


# ======================================== Get a joint angle  ======================================== #
# result, theta1 = vrep.simxGetJointPosition(clientID, handles[0][1], vrep.simx_opmode_blocking)
# if result != vrep.simx_return_ok:
#     raise Exception('could not get 1 joint variable')
#
# print(theta1)
#
# print(vrep.getJointMode(handles[0][0]))

[[17, 19, 21], [28, 30, 32], [38, 40, 42], [48, 50, 52], [58, 60, 62], [68, 70, 72]] [24, 35, 45, 55, 65, 75, 27]


In [98]:
# Close all open connections (Clear bad cache)
vrep.simxFinish(-1)
# Connect to V-REP (raise exception on failure)
clientID = vrep.simxStart('127.0.0.1', 19997, True, True, 5000, 5)
if clientID == -1:
	raise Exception('Failed connecting to remote API server')

# Start simulation
vrep.simxStartSimulation(clientID, vrep.simx_opmode_oneshot)

# ******************************** Your robot control code goes here  ******************************** #
#print(get_joint(handles, body_handles))
time.sleep(4)


#vrep.simxSetJointTargetPosition(clientID, handles[5][2], 0, vrep.simx_opmode_oneshot)


# for x in handles:
#     for y in x:
#         vrep.simxSetJointTargetPosition(clientID, y, 0, vrep.simx_opmode_oneshot)
# time.sleep(2)
    
for i in range(15):
    step_3([0.13086048,-0.13086048,-0.03120471])
# for i in range(20):
#     vrep.simxSetJointTargetPosition(clientID, handles[0][1], -1, vrep.simx_opmode_oneshot)
#     vrep.simxSetJointTargetPosition(clientID, handles[5][1], -1, vrep.simx_opmode_oneshot)
#     time.sleep(.2)
#     vrep.simxSetJointTargetPosition(clientID, handles[0][2], 2, vrep.simx_opmode_oneshot)
#     vrep.simxSetJointTargetPosition(clientID, handles[5][2], 2, vrep.simx_opmode_oneshot)
#     time.sleep(.2)
#     vrep.simxSetJointTargetPosition(clientID, handles[0][1], 0, vrep.simx_opmode_oneshot)
#     vrep.simxSetJointTargetPosition(clientID, handles[5][1], 0, vrep.simx_opmode_oneshot)
#     time.sleep(.2)
#     vrep.simxSetJointTargetPosition(clientID, handles[0][2], 0, vrep.simx_opmode_oneshot)
#     vrep.simxSetJointTargetPosition(clientID, handles[5][2], 0, vrep.simx_opmode_oneshot)
#     time.sleep(.2)
# time.sleep(2)


# Stop simulation
vrep.simxStopSimulation(clientID, vrep.simx_opmode_oneshot)
# Before closing the connection to V-REP, make sure that the last command sent out had time to arrive. You can guarantee this with (for example):
vrep.simxGetPingTime(clientID)
# Close the connection to V-REP
vrep.simxFinish(clientID)
print("==================== ** Simulation Ended ** ====================")

# ======================================================================================================= #
# ======================================== End Simulation =============================================== #
# ======================================================================================================= #

[0.7853981633974483, -0.785398143697901, 1.5707963408413999] [0.7853981633974483, -0.785398143697901, 1.9707963408414] [0, -0.785398143697901, 1.9707963408414]
step
[0.7853981633974483, -0.785398143697901, 1.5707963408413999] [0.7853981633974483, -0.785398143697901, 1.9707963408414] [0, -0.785398143697901, 1.9707963408414]
step
[0.7853981633974483, -0.785398143697901, 1.5707963408413999] [0.7853981633974483, -0.785398143697901, 1.9707963408414] [0, -0.785398143697901, 1.9707963408414]
step
[0.7853981633974483, -0.785398143697901, 1.5707963408413999] [0.7853981633974483, -0.785398143697901, 1.9707963408414] [0, -0.785398143697901, 1.9707963408414]
step
[0.7853981633974483, -0.785398143697901, 1.5707963408413999] [0.7853981633974483, -0.785398143697901, 1.9707963408414] [0, -0.785398143697901, 1.9707963408414]
step
[0.7853981633974483, -0.785398143697901, 1.5707963408413999] [0.7853981633974483, -0.785398143697901, 1.9707963408414] [0, -0.785398143697901, 1.9707963408414]
step
[0.7853981

In [41]:
a1 = np.array([0.08000001311302185, 0.0, 0.003102347254753113])
a2 = np.array([0.130264550447464, -5.960464477539063e-08, -0.00809653103351593])
a3 = np.array([0.19294512271881104, -5.960464477539063e-08, 0.028100647032260895])
a4 = np.array([0.20402440428733826, -5.125999450683594e-06, -0.08788306266069412])

l2 = np.linalg.norm(a2-a1)
print(l2)
l3 = np.linalg.norm(a3-a2)
print(l3)
le = np.linalg.norm(a4-a3)
print(le)

0.05149697649731044
0.07238155732083357
0.11651167932822327
